In [1]:
import numpy as np
import pandas as pd 
import csv

In [2]:
data_root = '/Users/svloaner/Desktop/yelp/'

train_photo_to_biz = pd.read_csv(data_root+'train_photo_to_biz_ids.csv')
train_df = pd.read_csv(data_root+ 'train_feature_pca.csv')
train_df1 = pd.merge(train_photo_to_biz, train_df, how='inner', on='photo_id')
train_df2 = train_df1.groupby('business_id')


In [47]:
feature_column = train_df.columns[:-1]
#feature_column1 = ['intesect-' + i for i in feature_column.values]
#feature_column2 = ['union-' + i for i in feature_column.values]
#feature_column3 = ['median-' + i for i in feature_column.values]


out_colum_name = ['business_id']
out_colum_name.extend(feature_column.values)
#out_colum_name.extend(feature_column1)
#out_colum_name.extend(feature_column2)
#out_colum_name.extend(feature_column3)

def getOneRow(name, grouped_df, train=1):
    out_list = [name]
    for f in feature_column:
        avg = np.mean(grouped_df[f])
        out_list.append(avg)
 #   for f in feature_column:
 #       intersection = np.min(grouped_df[f])
 #       out_list.append(intersection)
 #   for f in feature_column:
 #       union = np.max(grouped_df[f])
 #       out_list.append(union)
 #   for f in feature_column:
 #       median = np.median(grouped_df[f])
 #       out_list.append(median)
        
    return out_list

In [48]:
train_out_handle = open('train_business_feature1.csv', "w")
train_writer = csv.writer(train_out_handle)
train_writer.writerow(out_colum_name)
counter = 0
for name, group in train_df2:
    out_row = getOneRow(name, group, train=1)
    assert len(out_row) == len(out_colum_name)
    train_writer.writerow(out_row)
    counter += 1
    if counter%10000 == 0:
        print counter
train_out_handle.close()

In [49]:
train_business_feature = pd.read_csv(data_root+'train_business_feature1.csv')
train_label = pd.read_csv(data_root+'train.csv')
train_df3 = pd.merge(train_business_feature, train_label, how='inner', on='business_id')

In [50]:
train_df3['labels'].fillna('', inplace=True)
y_train = np.array([y.split() for y in train_df3['labels']])
y_train = [map(int, y) for y in y_train]

In [51]:
train_business_feature.drop('business_id', axis=1, inplace=True)

In [52]:
from sklearn import svm, datasets
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import multiprocessing

cores=multiprocessing.cpu_count()-2

mlb = MultiLabelBinarizer()
y_ptrain_mlb= mlb.fit_transform(y_train) 

random_state = np.random.RandomState(0)
X_ptrain, X_ptest, y_ptrain, y_ptest = train_test_split(train_business_feature, y_ptrain_mlb, 
                 test_size=.2,random_state=random_state)
print X_ptrain.shape, X_ptest.shape, y_ptrain.shape, y_ptest.shape

(1600, 100) (400, 100) (1600, 9) (400, 9)


In [120]:
classifier = OneVsRestClassifier(svm.SVC(kernel='rbf', C= 1.0, probability=True), n_jobs=cores)
classifier.fit(X_ptrain, y_ptrain)

OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=6)

In [121]:
y_ppredict = classifier.predict(X_ptest)

In [122]:
from sklearn.metrics import f1_score

print "F1 score: ", f1_score(y_ptest, y_ppredict, average='micro') 
print "Individual Class F1 score: ", f1_score(y_ptest, y_ppredict, average=None)

F1 score:  0.849423022798
Individual Class F1 score:  [ 0.74897119  0.85245902  0.88636364  0.68380463  0.80161943  0.90458015
  0.94845361  0.75486381  0.8963964 ]


In [123]:
#########test
test_photo_to_biz = pd.read_csv(data_root+'test_photo_to_biz.csv')
test_df = pd.read_csv(data_root+ 'test_feature_pca.csv')
test_df1 = pd.merge(test_photo_to_biz, test_df, how='inner', on='photo_id')
test_df2 = test_df1.groupby('business_id')
feature_column = test_df.columns[:-1]
test_out_handle = open('test_business_feature1.csv', "w")
test_writer = csv.writer(test_out_handle)
test_writer.writerow(out_colum_name)
counter = 0
for name, group in test_df2:
    out_row = getOneRow(name, group, train=0)
    assert len(out_row) == len(out_colum_name)
    test_writer.writerow(out_row)
    counter += 1
    if counter%10000 == 0:
        print counter
test_out_handle.close()

##################

10000


In [124]:
classifier.fit(train_business_feature, y_ptrain_mlb)

test_business_feature = pd.read_csv(data_root+'test_business_feature1.csv')
business_id = test_business_feature['business_id'].reshape(-1,1)
test_business_feature.drop('business_id', axis=1, inplace=True)
y_predict_test = classifier.predict(test_business_feature)

In [126]:
y_predict_label = mlb.inverse_transform(y_predict_test)

df = pd.DataFrame(columns=['business_id','labels'])

for i in range(len(y_predict_label)):
    biz = business_id[i][0]
    label = y_predict_label[i]
    label = str(label)[1:-1].replace(",", " ")
    df.loc[i] = [str(biz), label]

with open(data_root+"submission1.csv",'w') as f:
    df.to_csv(f, index=False) 